In [1]:
import geopandas as gpd
import tqdm
import requests
import json

areas = gpd.read_file(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\\2.merge_data\output4.geojson')

# extract geometry and title
areas_gdf = areas[['geometry', 'title']]

# distinct 
areas_gdf = areas_gdf.drop_duplicates()
# change crs
areas_gdf = areas_gdf.to_crs(epsg=4326)
# find centroid
areas_gdf['centroid'] = areas_gdf['geometry'].centroid

# extract lat and lon
areas_gdf['lat'] = areas_gdf['centroid'].apply(lambda x: x.y)
areas_gdf['lon'] = areas_gdf['centroid'].apply(lambda x: x.x)

# round to 2 decimal places
areas_gdf['lat'] = areas_gdf['lat'].round(2)
areas_gdf['lon'] = areas_gdf['lon'].round(2)

# reset index
areas_gdf = areas_gdf.reset_index(drop=True)

for i in tqdm.tqdm(range(len(areas_gdf))):
   url = 'https://rest.isric.org/soilgrids/v2.0/properties/query?lon={}&lat={}&property=bdod&depth=0-5cm&value=mean'.format(areas_gdf['lon'][i], areas_gdf['lat'][i])
   response = requests.get(url)
   data = json.loads(response.text)
   if 'properties' in data:
   # deal with missing data
         if data['properties']['layers'] == []:
            areas_gdf.loc[i, 'soil'] = None
            continue
         areas_gdf.loc[i, 'soil'] = data['properties']['layers'][0]['depths'][0]['values']['mean']

# change crs
areas = areas.to_crs(epsg=4326)

# merge on geometry
areas_gdf_new = areas_gdf.merge(areas, on='geometry', how='right')
# drop title_x centroid lat lon, rename soil_x to soil_quality,rename title_y to title, rename soil_y to soil, rename value to chirps_rainfall
areas_gdf_new = areas_gdf_new.drop(['title_x', 'centroid', 'lat', 'lon'], axis=1)
areas_gdf_new = areas_gdf_new.rename(columns={'soil_x': 'soil_quality', 'title_y': 'title', 'soil_y': 'soil', 'value': 'chirps_rainfall'})


# write to geojson
areas_gdf_new.to_file(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\\2.merge_data\output5.geojson', driver='GeoJSON')

C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_20504\2439317317.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas_gdf['centroid'] = areas_gdf['geometry'].centroid
100%|██████████| 298/298 [01:39<00:00,  2.99it/s]
